In [1]:
import csv
print(csv.__file__)

import os
import shutil
import pandas as pd
import numpy as np
from pandas import read_csv
from numpy import genfromtxt
from io import StringIO


/Users/soniasharapova/miniconda3/lib/python3.9/csv.py


# RNA

In [4]:
def downloadRnaData(fname, **kwargs):
    def rewrite_csv_as_tab(fname):
        with open(fname, newline='') as fp:
            dialect = csv.Sniffer().sniff(fp.readline())

            fp.seek(0)
            for row in csv.reader(fp, dialect):
                yield "\t".join(row)
    return np.recfromcsv(rewrite_csv_as_tab(fname), delimiter='\t', encoding=None, **kwargs, skip_header=5)

def downloadClinicalData(fname, **kwargs):
    def rewrite_csv_as_tab(fname):
        with open(fname, newline='') as fp:
            dialect = csv.Sniffer().sniff(fp.readline())
            fp.seek(0)
            for row in csv.reader(fp, dialect):
                yield "\t".join(row)
    return np.recfromcsv(rewrite_csv_as_tab(fname), delimiter='\t', encoding=None, **kwargs)

def testOpenFiles(directory):
    for root,dirs,files in os.walk(directory):
        files = [f for f in files if not f[0] == '.']
        dirs[:] = [d for d in dirs if not d[0] == '.']
        for file in files:
            path = directory + '/' + file
            print(path)
            #patient.append(file[0:file.find('.')])
            
            #openFile = pd.read_csv(path,sep='\t', header=5)
            downloadRnaData(path, case_sensitive=True)
            downloadClinicalData(path, case_sensitive=True)
            
            
def getPatientVector(directory, csv):
    data=downloadRnaData(csv)
    features=[]
    features.append('patient_id')
    
    #for root,dirs,files in os.walk(directory):
        #for file in files:
            #if file.endswith(".tsv"):
                #path = directory + '/' + file
                #features.append(file)
    for row in data:
        features.append(row[1])
    return(features)

def getCaseID(samplesheet, filename):
    caseID = 0
    samplesheet = downloadClinicalData(samplesheet)
    for row in samplesheet:
        #print(row.iloc[1])
        if row[1] == filename:
            caseID = row[5]
    return caseID


def findPatient(clinicalFile, patient):
    clinicalData = downloadClinicalData(clinicalFile)
    for row in clinicalData:
        if (row[1] == patient):
            return row[26]
    return None
            
def writeToCsv(outfile, infile):
    List_columns = infile[0]
    infile = infile[1:]
    df = pd.DataFrame(infile)
    df.to_csv(outfile, index=False, header=List_columns)

def fillMatrix(directory, csv, samplesheet):
    features = getPatientVector(directory, csv) 
    matrix = []
    matrix.append(features)
    #row = []

    
    for root,dirs,files in os.walk(directory):
        files = [f for f in files if not f[0] == '.']
        dirs[:] = [d for d in dirs if not d[0] == '.']
        for file in files:
            if file.endswith(".tsv"):
                print(file)
                row = []
                path = directory + '/' + file
                openFile = downloadRnaData(path, case_sensitive=True)
            
                patientID = getCaseID(samplesheet, file)
                row.append(patientID)
                for x in openFile:
                    row.append(x[7])
                
                matrix.append(row)
            #row.clear()
            
        #print(matrix)
        #os.makedirs('/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq', exist_ok=True)  
        #matrix.to_csv('/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/outTest.csv')
        
        #matrix.to_csv('/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/outTest.csv', index = False)

        
        
    
        print(matrix)
        return(matrix)
    
    
def addYforPatient(genePatientMatrix, clinicalFile, outfile):

    for row in genePatientMatrix:
        patientID = row[0]
        findPat = findPatient(clinicalFile, patientID)
        row.append(findPat)
        
    writeToCsv(outfile, genePatientMatrix)
    return genePatientMatrix
    
    
    
    
    

In [5]:
os.getcwd()
os.chdir("../../..")
os.getcwd()
os.listdir()
os.chdir("/Volumes/Toshiba_blue")
os.listdir()
os.getcwd()

'/Volumes/Toshiba_blue'

In [6]:
os.getcwd()
rnaFolder = './rnaSeq/rnaSeqFiles/files'
clinical = './clinical.tsv'
sample_sheet = './rnaSeq/rnaSeqSampleSheet.tsv'
#directoryRna = os.path.join("c:\\",rnaFolder)
testrnaFolder = './rnaSeq/rnaSeqFiles/files/0a9e33db-2527-4cc3-8669-a7c10fed7a7f.rna_seq.augmented_star_gene_counts.tsv'

#testrnaFolder = "/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/testSet/newTest/0f382cf1-3e52-4078-bf12-cc6ba998ed3f.rna_seq.augmented_star_gene_counts.tsv"

#test =          '/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/testSet/newTest/1a3dcc95-64bf-4886-9807-9511b24f30af.rna_seq.augmented_star_gene_counts.tsv'

#patients = getPatientID(directoryRna)
#sample_sheet = '/Users/soniasharapova/Downloads/rnaSeq_sample_sheet.tsv'
#outfile = './rnaSeq/rnaSeqFiles/outFile.csv'
outfile = './rnaSeq/rnaSeqFiles/outFileLymph.csv'
matrix = fillMatrix(rnaFolder, testrnaFolder, sample_sheet)
fullMatrix = addYforPatient(matrix, clinical, outfile)

print(fullMatrix)
#addYforPatient(matrix, clinical)

00063026-2a67-41de-b105-f80dca277978.rna_seq.augmented_star_gene_counts.tsv
00a26384-1b1c-4db4-9664-75fb9b3febdb.rna_seq.augmented_star_gene_counts.tsv
01661d94-fc16-4456-95cf-a5fa4e1e196c.rna_seq.augmented_star_gene_counts.tsv
01900aab-4c12-4198-b41f-ff638e5abe3b.rna_seq.augmented_star_gene_counts.tsv
01ae5613-1c58-4d02-8fd3-47d35d71c22d.rna_seq.augmented_star_gene_counts.tsv
01f6286b-8422-4f16-bec3-871c86396790.rna_seq.augmented_star_gene_counts.tsv
021d9e06-9d27-400c-8776-08e89c817b46.rna_seq.augmented_star_gene_counts.tsv
02423743-43b4-494b-acd9-1cd3bcd3d395.rna_seq.augmented_star_gene_counts.tsv
02a4ccea-1464-4664-bc44-bb6569a34608.rna_seq.augmented_star_gene_counts.tsv
02a50d1b-ff62-4936-a0ce-ac0e22a5b41c.rna_seq.augmented_star_gene_counts.tsv
02bbe1b1-44ff-426b-a40e-47a01bc4cf7d.rna_seq.augmented_star_gene_counts.tsv
032e5aa7-8028-48ad-8fe4-6a5cc6aee5cb.rna_seq.augmented_star_gene_counts.tsv
034f8d01-f664-494b-bc04-77485000611f.rna_seq.augmented_star_gene_counts.tsv
0361ec21-e83

1516c4ec-98c0-41dc-9ad1-86cc05305fc1.rna_seq.augmented_star_gene_counts.tsv
151af32a-7367-4e35-a575-f6da31aa256a.rna_seq.augmented_star_gene_counts.tsv
158ab1d9-8925-4a05-95da-b2e0ca297474.rna_seq.augmented_star_gene_counts.tsv
15933e60-dee1-493a-a4f9-d5791dd15325.rna_seq.augmented_star_gene_counts.tsv
16bd1ebb-5bf9-4839-99bd-5a4d3dcad7b8.rna_seq.augmented_star_gene_counts.tsv
16e1fcb7-863d-40c1-b2d6-365d519148f6.rna_seq.augmented_star_gene_counts.tsv
171e095d-2236-43c8-9670-23608e6cc8ab.rna_seq.augmented_star_gene_counts.tsv
1729f18b-f4d7-4e1c-af9f-2c651023e7fc.rna_seq.augmented_star_gene_counts.tsv
1742c1d9-a359-4d9e-b1e4-1d162472e67a.rna_seq.augmented_star_gene_counts.tsv
17985496-652a-4034-997b-b888d97316e3.rna_seq.augmented_star_gene_counts.tsv
17dfa221-f092-45ed-a2e4-5798ddfc0ce9.rna_seq.augmented_star_gene_counts.tsv
17efe3f1-a97a-4e26-b28a-98702fd3c42b.rna_seq.augmented_star_gene_counts.tsv
18294571-5fc3-4e68-8b31-5fc6a207639f.rna_seq.augmented_star_gene_counts.tsv
187f14ba-000

2dd0b954-db3e-402d-9c06-6160e199dedb.rna_seq.augmented_star_gene_counts.tsv
2df3cde4-5d53-4e2c-829a-e609f1d92a7b.rna_seq.augmented_star_gene_counts.tsv
2e1410fb-4799-4a85-97e2-0d05c9255977.rna_seq.augmented_star_gene_counts.tsv
2e36839f-3155-4e05-9ed3-404522fdb04f.rna_seq.augmented_star_gene_counts.tsv
2e41a573-7a9d-4f96-873f-48aa12b3ab96.rna_seq.augmented_star_gene_counts.tsv
2e4316d2-5fe5-4ee1-9b22-73f4c42b6202.rna_seq.augmented_star_gene_counts.tsv
2e64abe2-6024-4d28-9e09-560ce2a9fd15.rna_seq.augmented_star_gene_counts.tsv
2ed88c3e-bf61-4e2f-b06c-0b93c9f9402e.rna_seq.augmented_star_gene_counts.tsv
2f05f114-9769-48e4-978c-6bfe1f2f372f.rna_seq.augmented_star_gene_counts.tsv
2f21d781-2df5-4de1-a595-a61507cfe610.rna_seq.augmented_star_gene_counts.tsv
2f3e03a2-8764-4791-b53a-a599c6159e19.rna_seq.augmented_star_gene_counts.tsv
2f51534b-248b-4999-bc3f-e42a2e98332e.rna_seq.augmented_star_gene_counts.tsv
30285113-d411-475a-a8ed-1fb66be72f28.rna_seq.augmented_star_gene_counts.tsv
304b7293-0a7

KeyboardInterrupt: 

# miRNA

In [7]:
def downloadMiRnaData(fname, **kwargs):
    def rewrite_csv_as_tab(fname):
        with open(fname, newline='') as fp:
            dialect = csv.Sniffer().sniff(fp.readline())

            fp.seek(0)
            for row in csv.reader(fp, dialect):
                yield "\t".join(row)
    return np.recfromcsv(rewrite_csv_as_tab(fname), delimiter='\t', encoding=None, **kwargs)

def downloadClinicalData(fname, **kwargs):
    def rewrite_csv_as_tab(fname):
        with open(fname, newline='') as fp:
            dialect = csv.Sniffer().sniff(fp.readline())
            fp.seek(0)
            for row in csv.reader(fp, dialect):
                yield "\t".join(row)
    return np.recfromcsv(rewrite_csv_as_tab(fname), delimiter='\t', encoding=None, **kwargs)

def testOpenFiles(directory):
    for root,dirs,files in os.walk(directory):
        files = [f for f in files if not f[0] == '.']
        dirs[:] = [d for d in dirs if not d[0] == '.']
        for file in files:
            path = directory + '/' + file
            #print(path)
            #patient.append(file[0:file.find('.')])
            
            #openFile = pd.read_csv(path,sep='\t', header=5)
            downloadMiRnaData(path, case_sensitive=True)
            downloadClinicalData(path, case_sensitive=True)
            co
    
    
    
    
def getUniqueID(data):
    uniqueArray = []
    for row in data:
        ident = row[0]
        if ident not in uniqueArray:
            uniqueArray.append(ident)
    return uniqueArray
            
            
            
def getPatientVector(directory, csv):
    data=downloadMiRnaData(csv)
    features=[]
    features.append('patient_id')
    features.extend(getUniqueID(data))
    
    return(features)




def getCaseID(samplesheet, filename):
    caseID = 0
    samplesheet = downloadClinicalData(samplesheet)
    for row in samplesheet:
        #print(row.iloc[1])
        
        if row[1] == filename:
            #print(row[1])
            print(filename)
            caseID = row[5]
    return caseID



def miRnaToGeneName(samplesheet, miRnaID):
    caseID = 0
    samplesheet = downloadClinicalData(samplesheet)
    for row in samplesheet:
        if row[1] == miRnaID:
            caseID = row[1]
    return caseID


def findPatientY(clinicalFile, patient):
    clinicalData = downloadClinicalData(clinicalFile)
    for row in clinicalData:
        if (row[1] == patient):
            
            return row[26]
    print(row[1],patient)
    return None

def getAvgReadCount(unique, csvData):
    sumArray = []
    
    for ident in unique:
        summ = 0
        count = 1
        for row in csvData:
            if row[0] == ident:
                summ = row[2] + summ
                count = count + 1
        if count == 1:
            avg = int(summ / count)
        else:
            avg = int(summ / (count - 1) )
        sumArray.append(avg)
            
    return sumArray



def getReadCountMirna(csvData):
    readCount = []
    count = 0
    for row in csvData:
        count = row[1]
        readCount.append(count)
        count = 0
    return readCount
            
def writeToCsv(outfile, infile):
    List_columns = infile[0]
    infile = infile[1:]
    df = pd.DataFrame(infile)
    df.to_csv(outfile, index=False, header=List_columns)

def fillMatrix(directory, csv, samplesheet):
    features = getPatientVector(directory, csv) 
    matrix = []
    matrix.append(features)
    #row = []
    openCSV = downloadMiRnaData(csv, case_sensitive=True)
    unique = getUniqueID(openCSV)
    
    for root,dirs,files in os.walk(directory):
        files = [f for f in files if not (f[0] == '.' or f == 'annotations.txt' or not f.endswith(".txt"))]
        dirs[:] = [d for d in dirs if not d[0] == '.']
        for file in files:
            row = []
            path = directory + '/' + file
            openFile = downloadMiRnaData(path, case_sensitive=True)
            patientID = getCaseID(samplesheet, file)
            row.append(patientID)
            #row.extend(getAvgReadCount(unique, openFile))
            row.extend(getReadCountMirna(openFile))
            matrix.append(row)
    
        print(matrix)
        return(matrix)
    
    
def addYforPatient(genePatientMatrix, clinicalFile, outfile):
    line = 0
    for row in genePatientMatrix:
        line = line+1
        print(line)
        patientID = row[0]
        findPat = findPatientY(clinicalFile, patientID)
        print('findPat',row[0], findPat)
        row.append(findPat)
        
    writeToCsv(outfile, genePatientMatrix)
    return genePatientMatrix
    
    
    
    
    

In [8]:
os.getcwd()
os.chdir("../../..")
os.getcwd()
os.listdir()
os.chdir("/Volumes/Toshiba_blue")
os.listdir()
os.getcwd()

'/Volumes/Toshiba_blue'

In [9]:
os.getcwd()
miRnaFolder = './miRnaSeq/miRnaSeqFiles/mirnaFilesOnly'
clinical = './clinical.tsv'
sample_sheet = './miRnaSeq/miRnaSeqSampleSheet.tsv'
#directoryRna = os.path.join("c:\\",rnaFolder)
testMiRnaFolder = './miRnaSeq/miRnaSeqFiles/files/0a00ab98-e267-45f9-89af-5fecf427aca9.mirbase21.mirnas.quantification.txt'

#testrnaFolder = "/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/testSet/newTest/0f382cf1-3e52-4078-bf12-cc6ba998ed3f.rna_seq.augmented_star_gene_counts.tsv"

#test =          '/Users/soniasharapova/Documents/JunDingResearch/MultiOmics/rnaSeq/testSet/newTest/1a3dcc95-64bf-4886-9807-9511b24f30af.rna_seq.augmented_star_gene_counts.tsv'

#patients = getPatientID(directoryRna)
#sample_sheet = '/Users/soniasharapova/Downloads/rnaSeq_sample_sheet.tsv'
miRnaOutfile = './miRnaSeq/miRnaSeqFiles/outFileLymph.csv'
miRnaMatrix = fillMatrix(miRnaFolder, testMiRnaFolder, sample_sheet)



004b1938-87ab-4bf2-a19b-d2953d0e81a0.mirbase21.mirnas.quantification.txt
00574710-29e7-4df3-a26d-7cdfd19aef31.mirbase21.mirnas.quantification.txt
00cdf835-e345-4027-a68e-64db83d1a01f.mirbase21.mirnas.quantification.txt
010de2d6-ab41-4779-923a-1c288296e267.mirbase21.mirnas.quantification.txt
0139dd64-61be-43c9-a0cd-b1a2d2b2196c.mirbase21.mirnas.quantification.txt
014ac73d-9fdb-43f9-8495-8dab01f24859.mirbase21.mirnas.quantification.txt
014baf8e-92f0-4f45-8c1d-cfa6be214f97.mirbase21.mirnas.quantification.txt
01570544-ec92-45ed-b59a-5acea638eda7.mirbase21.mirnas.quantification.txt
01629488-8b88-47fc-804d-dd5b78191c87.mirbase21.mirnas.quantification.txt
01d9dfa7-e2e8-48d3-810a-bbf3a371528c.mirbase21.mirnas.quantification.txt
01e271f0-6fca-4d52-9cb9-d080d0918337.mirbase21.mirnas.quantification.txt
0206f9ba-7445-490f-8afb-2fc66fda8c56.mirbase21.mirnas.quantification.txt
0266681b-6a04-4dc4-a1be-296a42ab33d4.mirbase21.mirnas.quantification.txt
0296f7dd-e800-4959-9e09-2efddd54f60c.mirbase21.mirn

17734bbe-d8b7-43d8-abe1-9fad8468c264.mirbase21.mirnas.quantification.txt
17baa13d-9eca-41cf-864c-b26978d8a563.mirbase21.mirnas.quantification.txt
1806dee6-8114-4435-8eb8-ea49399abbe3.mirbase21.mirnas.quantification.txt
1809f3c6-7c76-4b0c-bd18-ce8a1468b07b.mirbase21.mirnas.quantification.txt
182bade8-48cd-4072-8f65-9b248700ff36.mirbase21.mirnas.quantification.txt
183b24a1-dc9a-4078-bf01-4633823704d0.mirbase21.mirnas.quantification.txt
184deddc-0e07-46d4-bf99-5ad82e446300.mirbase21.mirnas.quantification.txt
1868d917-37e4-4607-8162-1027cfed8dd7.mirbase21.mirnas.quantification.txt
187a136b-89c4-4094-b363-5af5cbc92098.mirbase21.mirnas.quantification.txt
18d4f9c3-6002-4cf8-a13e-c94b63fa44a5.mirbase21.mirnas.quantification.txt
18ed9c00-b031-4f59-bb32-874da11ef509.mirbase21.mirnas.quantification.txt
19475873-d8f4-4954-8339-d0f9f1ab2bdc.mirbase21.mirnas.quantification.txt
19817650-f66f-4eb9-8d97-016448863b56.mirbase21.mirnas.quantification.txt
19872037-837e-490b-9ea4-16ddfd9e1c2b.mirbase21.mirn

31268895-fd75-451e-8044-e1223555b79c.mirbase21.mirnas.quantification.txt
3163c420-eccd-4bf2-9bb6-170f3549556c.mirbase21.mirnas.quantification.txt
316b1936-e040-4ea3-8346-d00b2b98cf0a.mirbase21.mirnas.quantification.txt
31ce1d57-ce67-4c6f-bd4c-0fbcbf9cdbe5.mirbase21.mirnas.quantification.txt
31ce585a-95d2-4b0b-b9cd-d4f86f994ef0.mirbase21.mirnas.quantification.txt
31edc83e-fd81-4c00-85f0-396701cbc0b5.mirbase21.mirnas.quantification.txt
31fa9bfd-dc91-4e27-be93-db7df8df2cd8.mirbase21.mirnas.quantification.txt
322dab3b-6516-45af-b25e-00524b73e861.mirbase21.mirnas.quantification.txt
331cccd2-f121-4ba8-80b9-b53f369b8add.mirbase21.mirnas.quantification.txt
3346e3c2-0458-4879-9522-d8c427b02b8c.mirbase21.mirnas.quantification.txt
33b9e3dd-1553-408a-81b0-7552bb1e7305.mirbase21.mirnas.quantification.txt
33cd75d7-57c9-427a-bbee-4e009f29f318.mirbase21.mirnas.quantification.txt
342bbbcb-5c42-4c9b-a779-f5925004a889.mirbase21.mirnas.quantification.txt
3478251c-ab08-4802-8384-a52c06dad91f.mirbase21.mirn

4edc18e9-dc60-4c91-a648-4e3e4652ef9b.mirbase21.mirnas.quantification.txt
4ee53bf1-9bfd-405c-a166-c88e665ab967.mirbase21.mirnas.quantification.txt
4f3dbf34-7147-4ae2-a874-f6755480a72a.mirbase21.mirnas.quantification.txt
4f569ea7-92d9-4a29-9f0e-8a5bfbdd717f.mirbase21.mirnas.quantification.txt
4f80ab99-6719-4626-92bc-c90a1b1ed724.mirbase21.mirnas.quantification.txt
4f9ab944-ee29-4f43-ac5d-61cee10483ac.mirbase21.mirnas.quantification.txt
4fbe4ee1-7efd-4ba4-8116-5ab3b4666b48.mirbase21.mirnas.quantification.txt
4fbfc232-818f-48fb-83bc-6433c9207b09.mirbase21.mirnas.quantification.txt
4fc53a6e-e97e-42fb-9544-da561d30ea06.mirbase21.mirnas.quantification.txt
504d9163-bf77-4ad0-8db9-dd2b7853e024.mirbase21.mirnas.quantification.txt
50879004-d3d4-4345-aab1-24a9ef0ac405.mirbase21.mirnas.quantification.txt
508ba40f-1ef5-49ac-9dcf-6c32680a3338.mirbase21.mirnas.quantification.txt
508f1179-38e7-4ece-aabe-2c0c654f52d3.mirbase21.mirnas.quantification.txt
513a912d-ce2b-45b6-9880-f1d17dfc2d66.mirbase21.mirn

636c9329-44fb-4dd4-b0d6-2bf0bc9531de.mirbase21.mirnas.quantification.txt
636e09a5-2922-48b9-8aab-875fbba5628d.mirbase21.mirnas.quantification.txt
6380cd41-2628-484f-acd3-2240d9903fba.mirbase21.mirnas.quantification.txt
640f2035-1d24-4f16-bd8d-0017fad45fa6.mirbase21.mirnas.quantification.txt
64302b1c-cfec-4d76-a67c-5455c68dbb89.mirbase21.mirnas.quantification.txt
643cef15-19c3-4317-bb16-7347c7c14b6e.mirbase21.mirnas.quantification.txt
644cbd72-9af7-4387-830e-ba7542cdde27.mirbase21.mirnas.quantification.txt
645290eb-8d7e-4342-abdc-b1bb1ff4b545.mirbase21.mirnas.quantification.txt
6490b269-cbb4-4576-bc30-b12470f3740e.mirbase21.mirnas.quantification.txt
64d93d29-2669-4ad4-bc6e-6bf6f2baeca2.mirbase21.mirnas.quantification.txt
650a7a01-faf2-43f3-924c-bdd7b9914294.mirbase21.mirnas.quantification.txt
651dd4aa-adf0-4218-ba4e-ff12756ebab8.mirbase21.mirnas.quantification.txt
6520f3db-0e25-4dac-8aae-d96da9712d85.mirbase21.mirnas.quantification.txt
6525a77a-cdff-456a-857d-ebf5ea6a880d.mirbase21.mirn

...

abb12646-a68e-4819-9b06-baea10617c8a.mirbase21.mirnas.quantification.txt
ac3ccdb6-84ee-43d7-afc4-8172e387b138.mirbase21.mirnas.quantification.txt
ac40e8b0-eb84-44eb-b2e1-12fe5d065f00.mirbase21.mirnas.quantification.txt
ac95539e-0d2f-4877-8e02-a12897f080e6.mirbase21.mirnas.quantification.txt
aca6f2fe-93bf-4e13-b634-4fa3390fd13c.mirbase21.mirnas.quantification.txt
acc69d43-989d-4ee9-a767-9470c099ad78.mirbase21.mirnas.quantification.txt
acf93050-5e7d-4db6-a47f-5b669e9e9b7a.mirbase21.mirnas.quantification.txt
ad2efe98-8719-45a1-8f14-3fd15c03af21.mirbase21.mirnas.quantification.txt
ad7d6f10-1dff-4bc8-a8c3-0b6ce7b45366.mirbase21.mirnas.quantification.txt
addaf3a8-26ce-45e8-b8b1-885f22e567a7.mirbase21.mirnas.quantification.txt
ae012200-cc65-40f3-bdae-7c7eadf157e2.mirbase21.mirnas.quantification.txt
ae26ba1b-2b15-4b4c-ba9a-2d7cddf5dda0.mirbase21.mirnas.quantification.txt
ae4877c3-2ab0-424b-b32f-42ea18a48b51.mirbase21.mirnas.quantification.txt
ae70879f-5cc0-43e5-b9ce-f3d7e4e54e0a.mirbase21.mirn

c4cea915-ca34-4dfc-a7f1-372ec285901f.mirbase21.mirnas.quantification.txt
c4e955ec-b9e8-4dc3-a5f0-f2468861f6e5.mirbase21.mirnas.quantification.txt
c507895a-d057-407b-897f-ca5e1009063c.mirbase21.mirnas.quantification.txt
c509bc64-ea7a-4f8a-957c-66539d086536.mirbase21.mirnas.quantification.txt
c520163c-e7d0-4f49-bfdf-0eae09169975.mirbase21.mirnas.quantification.txt
c530a730-ab95-4b97-9cbd-dc7249e087c7.mirbase21.mirnas.quantification.txt
c53c81db-0bad-4e9a-b320-75c51691f859.mirbase21.mirnas.quantification.txt
c53d19da-b426-473b-86e9-a596894fdbbb.mirbase21.mirnas.quantification.txt
c57795a9-4882-4f61-b903-7343166e071a.mirbase21.mirnas.quantification.txt
c58119b0-6b14-4d66-8a5a-fbc182ab5fe2.mirbase21.mirnas.quantification.txt
c58452d9-80ff-4fd7-aeed-cde69a675cfa.mirbase21.mirnas.quantification.txt
c5b3879a-c29a-4d5f-b0fa-fcfc3943443e.mirbase21.mirnas.quantification.txt
c5d4be4c-7b13-4c1c-9a2a-a8ee565da468.mirbase21.mirnas.quantification.txt
c5d93ed9-80d9-450a-9178-0fe28c59a633.mirbase21.mirn

dd3ab228-1c35-4710-aae3-68458bf959db.mirbase21.mirnas.quantification.txt
dd418c99-460d-45dd-9ca2-6fc22415d36d.mirbase21.mirnas.quantification.txt
dde887a7-33ba-4274-9583-45c20e41d213.mirbase21.mirnas.quantification.txt
de0ea0bb-a90d-4987-a6ec-8e7fc480724f.mirbase21.mirnas.quantification.txt
de1880cd-f294-4a59-9364-088b3af36725.mirbase21.mirnas.quantification.txt
de47d44c-221b-4663-8406-a2903ff64153.mirbase21.mirnas.quantification.txt
de5d906c-54e7-4822-988e-4820bca02d2b.mirbase21.mirnas.quantification.txt
def1c293-5928-4815-8474-ca4f25d0a678.mirbase21.mirnas.quantification.txt
df8fa00d-9f94-4526-8d6d-53b71ec7a145.mirbase21.mirnas.quantification.txt
dfb1fbb4-faab-4dc3-a7c2-9c2cd5838153.mirbase21.mirnas.quantification.txt
e00a6fae-7110-40cd-90d9-8622f8a51f4c.mirbase21.mirnas.quantification.txt
e01fdd5c-0cb2-41d4-9320-7e609f1ea91e.mirbase21.mirnas.quantification.txt
e13a1beb-e1d2-4f0d-a24d-018e131e3c93.mirbase21.mirnas.quantification.txt
e13d83d7-5eb9-4f01-b554-072c580cd183.mirbase21.mirn

f4cafe8e-81db-4c7e-aae4-765c52de2051.mirbase21.mirnas.quantification.txt
f4e000ed-f688-4bf1-8c80-eb30a9636cd2.mirbase21.mirnas.quantification.txt
f4e7c755-9d9f-4cbb-99e6-091d75be00ae.mirbase21.mirnas.quantification.txt
f50cb3c4-4f8d-415e-95a7-de2baab0daef.mirbase21.mirnas.quantification.txt
f51488b5-c09e-4f81-b255-3f59def8ef1d.mirbase21.mirnas.quantification.txt
f52852ac-43ed-477a-9967-96ac703638c5.mirbase21.mirnas.quantification.txt
f58e2a07-8aa8-4bc4-9fb5-7769ca79a2af.mirbase21.mirnas.quantification.txt
f59bd4e6-c154-4509-be7c-0aec2fb61875.mirbase21.mirnas.quantification.txt
f5a2f708-7821-47b6-95b8-66e804489335.mirbase21.mirnas.quantification.txt
f5bac553-9032-42ca-9a24-ee4e6d6ff0c1.mirbase21.mirnas.quantification.txt
f6ebca33-9e8d-4246-a191-d588985dbbed.mirbase21.mirnas.quantification.txt
f7c21de1-0018-4352-9f42-e883f2ef4009.mirbase21.mirnas.quantification.txt
f7f6549c-bede-4215-a497-9269beedb457.mirbase21.mirnas.quantification.txt
f846c0ad-9bbd-46a5-a678-fc3c05cdcc41.mirbase21.mirn

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
fullMatrix = addYforPatient(miRnaMatrix, clinical, outfile)

1
TCGA-E2-A14S patient_id
findPat patient_id None
2
findPat TCGA-AN-A0XV N2
3
findPat TCGA-OL-A5RW N1
4
findPat TCGA-E2-A15H N1mi
5
findPat TCGA-EW-A1PD N1a
6
findPat TCGA-EW-A1P5 N1a
7
findPat TCGA-A7-A26F N0
8
findPat TCGA-D8-A27K N1a
9
findPat TCGA-BH-A0AV N0
10
findPat TCGA-GM-A3XG N1a
11
findPat TCGA-E2-A1LI N1
12
findPat TCGA-A2-A4S0 N0
13
findPat TCGA-BH-A1F8 N2
14
findPat TCGA-D8-A1JE N1a
15
findPat TCGA-AR-A1AI N0
16
findPat TCGA-E2-A1BD N0
17
findPat TCGA-A8-A08G N0
18
findPat TCGA-A7-A26G N0 (i+)
19
findPat TCGA-BH-A0BL N0
20
findPat TCGA-E9-A1RD N0
21
findPat TCGA-E2-A1L8 N1a
22
findPat TCGA-BH-A0GZ N1a
23
findPat TCGA-AC-A62V N1a
24
findPat TCGA-BH-A18N N1b
25
findPat TCGA-AC-A3W7 N1a
26
findPat TCGA-AN-A0G0 N0
27
findPat TCGA-C8-A12L N0 (i-)
28
findPat TCGA-E2-A572 N2
29
findPat TCGA-A8-A08J N3
30
findPat TCGA-PE-A5DE N0
31
findPat TCGA-BH-A0BJ N1a
32
findPat TCGA-EW-A1P6 N1a
33
findPat TCGA-BH-A0B5 N2a
34
findPat TCGA-E9-A1RF N2
35
findPat TCGA-B6-A40C N0 (i-)
36
findPat

findPat TCGA-A2-A0CL N2a
286
findPat TCGA-E2-A1II N0
287
findPat TCGA-A2-A3XW N1a
288
findPat TCGA-A2-A3Y0 N1a
289
findPat TCGA-A2-A0YM N0 (i-)
290
findPat TCGA-BH-A0HB N0 (i-)
291
findPat TCGA-B6-A0IN NX
292
findPat TCGA-BH-A0DT N1a
293
findPat TCGA-BH-A0C3 N0
294
findPat TCGA-GI-A2C9 N0
295
findPat TCGA-AO-A129 N1a
296
findPat TCGA-AN-A0FX N0
297
findPat TCGA-BH-A0HY N0
298
findPat TCGA-A7-A4SC N0
299
findPat TCGA-BH-A1FE N1
300
findPat TCGA-A2-A3XU N1mi
301
findPat TCGA-B6-A0I6 N1
302
findPat TCGA-A2-A0EO N0 (i-)
303
findPat TCGA-E9-A5UP N0
304
findPat TCGA-OL-A6VQ N0
305
findPat TCGA-B6-A0IA N0 (i-)
306
findPat TCGA-BH-A0B5 N2a
307
findPat TCGA-AO-A03M N0 (i-)
308
findPat TCGA-A2-A0EU N0 (i-)
309
findPat TCGA-A2-A0YD N0 (i+)
310
findPat TCGA-BH-A0HI N0 (i-)
311
findPat TCGA-C8-A1HL N1
312
findPat TCGA-E9-A244 N0 (i-)
313
findPat TCGA-E2-A15I N0
314
findPat TCGA-D8-A27F N0
315
findPat TCGA-A8-A0A2 N0
316
findPat TCGA-E2-A10F N0 (i-)
317
findPat TCGA-EW-A424 N1
318
findPat TCGA-AO-A0

findPat TCGA-D8-A1XY N0
567
findPat TCGA-BH-A0BW N0
568
findPat TCGA-E9-A1N3 N2
569
findPat TCGA-D8-A27I N2a
570
findPat TCGA-GM-A2DF N1a
571
findPat TCGA-A7-A56D N0
572
findPat TCGA-BH-A1EX N1b
573
findPat TCGA-BH-A0DS N2a
574
findPat TCGA-A7-A6VY N1
575
findPat TCGA-B6-A0WS NX
576
findPat TCGA-AR-A24K N0
577
findPat TCGA-S3-AA11 N0
578
findPat TCGA-BH-A0AY N0 (i-)
579
findPat TCGA-A2-A4S2 N2
580
findPat TCGA-A7-A425 N3a
581
findPat TCGA-D8-A1JF N2a
582
findPat TCGA-BH-A0DO N0
583
findPat TCGA-C8-A138 N2
584
findPat TCGA-LL-A50Y N0
585
findPat TCGA-OK-A5Q2 N1a
586
findPat TCGA-B6-A0IH N1b
587
findPat TCGA-B6-A0WZ N1b
588
findPat TCGA-D8-A145 N1a
589
findPat TCGA-A2-A1FX N2a
590
findPat TCGA-EW-A1PA N1a
591
findPat TCGA-EW-A1IY N0
592
findPat TCGA-AC-A2FO N1
593
findPat TCGA-D8-A1JJ N0
594
findPat TCGA-LL-A6FQ N2a
595
findPat TCGA-AR-A256 N0
596
findPat TCGA-D8-A73W N1a
597
findPat TCGA-BH-A0C0 N1a
598
findPat TCGA-EW-A3E8 N0
599
findPat TCGA-A7-A0D9 N0 (i-)
600
findPat TCGA-A2-A0T0 N1

findPat TCGA-A7-A5ZX N3a
849
findPat TCGA-A8-A06P N2a
850
findPat TCGA-BH-A8FY N0 (i+)
851
findPat TCGA-C8-A3M8 N0
852
findPat TCGA-S3-A6ZF N0
853
findPat TCGA-BH-A0E9 N1a
854
findPat TCGA-C8-A8HQ N1
855
findPat TCGA-BH-A0DK N0 (i-)
856
findPat TCGA-A1-A0SP N0 (i-)
857
findPat TCGA-D8-A1J8 N1a
858
findPat TCGA-C8-A12Z N1
859
findPat TCGA-EW-A1OX N0
860
findPat TCGA-AO-A12G N0 (i+)
861
findPat TCGA-E9-A1RA N0
862
findPat TCGA-C8-A131 N2
863
findPat TCGA-A8-A086 N1a
864
findPat TCGA-E2-A14Z N0
865
findPat TCGA-AO-A0JG N2a
866
findPat TCGA-A2-A0CW N1a
867
findPat TCGA-D8-A1X6 N2a
868
findPat TCGA-E9-A22G N0
869
findPat TCGA-E9-A2JT N0 (i-)
870
findPat TCGA-D8-A27L N2a
871
findPat TCGA-C8-A12U N1
872
findPat TCGA-LL-A441 N0
873
findPat TCGA-AC-A3BB N2a
874
findPat TCGA-E9-A1R7 N0
875
findPat TCGA-BH-A18R N1
876
findPat TCGA-E2-A1IU N0 (mol+)
877
findPat TCGA-E9-A3QA N0 (i-)
878
findPat TCGA-A8-A090 N0
879
findPat TCGA-A7-A13H N1a
880
findPat TCGA-AR-A0U4 N0
881
findPat TCGA-D8-A1XG N1a
882

findPat TCGA-A8-A07C N0
1126
findPat TCGA-BH-A0BJ N1a
1127
findPat TCGA-BH-A1EW N1b
1128
findPat TCGA-B6-A402 N0 (i-)
1129
findPat TCGA-A8-A08X N3a
1130
findPat TCGA-A2-A0T5 N0 (i-)
1131
findPat TCGA-BH-A1FM N2
1132
findPat TCGA-BH-A0EI N1a
1133
findPat TCGA-D8-A27H N0
1134
findPat TCGA-A7-A3IZ N0 (i+)
1135
findPat TCGA-BH-A0BZ N1a
1136
findPat TCGA-A8-A0A7 N1a
1137
findPat TCGA-A7-A26H N1a
1138
findPat TCGA-D8-A1XD N2a
1139
findPat TCGA-C8-A27A N1
1140
findPat TCGA-UL-AAZ6 N0
1141
findPat TCGA-BH-A0DZ N1a
1142
findPat TCGA-E2-A570 N1mi
1143
findPat TCGA-AC-A3YJ N0 (i-)
1144
findPat TCGA-BH-A0W7 N0 (i-)
1145
findPat TCGA-EW-A1P8 N3b
1146
findPat TCGA-BH-A208 N1b
1147
findPat TCGA-D8-A27W N0
1148
findPat TCGA-AC-A6IX N3a
1149
findPat TCGA-E2-A1B6 N0
1150
findPat TCGA-AC-A62Y N1
1151
findPat TCGA-A2-A4RY N3a
1152
findPat TCGA-A8-A09Q N2a
1153
findPat TCGA-A2-A0CV N1a
1154
findPat TCGA-B6-A1KI N0
1155
findPat TCGA-EW-A1PF N0
1156
findPat TCGA-BH-A0HN N0 (i-)
1157
findPat TCGA-A2-A259 N0 (

In [11]:
miRnaOutfileMatrix = './miRnaSeq/miRnaSeqFiles/outFileLymph.csv'
outf = './miRnaSeq/miRnaSeqFiles/dummyOut.csv'
miRnaOutfileMatrixCSV = downloadMiRnaData(miRnaOutfileMatrix, case_sensitive=True)

#type(miRnaOutfileMatrixCSV)
miRnaOutfileMatrixCSV = pd.DataFrame(miRnaOutfileMatrixCSV).to_numpy()
type(miRnaOutfileMatrixCSV)
#arr.dtype()
#np.array(miRnaOutfileMatrixCSV.tolist())
#print(type(miRnaOutfileMatrixCSV))

#csv = writeToCsv(outf, miRnaOutfileMatrix)

FileNotFoundError: [Errno 2] No such file or directory: './miRnaSeq/miRnaSeqFiles/outFileLymph.csv'

In [ ]:
clinical = './clinical.tsv'
miRnaOutfile = './miRnaSeq/miRnaSeqFiles/outFileLymph.csv'

fullMatrix = addYforPatient(miRnaOutfileMatrixCSV, clinical, miRnaOutfile)

print(fullMatrix)